In [14]:
%matplotlib inline
import os, sys
import re
import string
import pathlib
import random
from collections import Counter, OrderedDict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from tqdm import tqdm, tqdm_notebook, tnrange
tqdm.pandas(desc='Progress')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torchtext
from torchtext import data
from torchtext import vocab

from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import accuracy_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
train_path = './train.csv'
test_path = './test.csv'
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [3]:
df_train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province as a nation in the 1960s?,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you encourage people to adopt and not shop?",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity affect space geometry?,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg hemispheres?,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain bike by just changing the tyres?,0


In [4]:
df_train.groupby('target').count()

,qid,question_text
target,,
0,1225312,1225312
1,80810,80810


In [5]:
df_train.where(df_train['target'] == 1).dropna()

,qid,question_text,target
22,0000e91571b60c2fb487,Has the United States become the largest dictatorship in the world?,1.0
30,00013ceca3f624b09f42,Which babies are more sweeter to their parents? Dark skin babies or light skin babies?,1.0
110,0004a7fcb2bf73076489,If blacks support school choice and mandatory sentencing for criminals why don't they vote Republican?,1.0
114,00052793eaa287aff1e1,"I am gay boy and I love my cousin (boy). He is sexy, but I dont know what to do. He is hot, and I want to see his di**. What should I do?",1.0
115,000537213b01fd77b58a,Which races have the smallest penis?,1.0
119,00056d45a1ce63856fc6,Why do females find penises ugly?,1.0
127,0005de07b07a17046e27,How do I marry an American woman for a Green Card? How much do they charge?,1.0
144,00068875d7c82a5bcf88,"Why do Europeans say they're the superior race, when in fact it took them over 2,000 years until mid 19th century to surpass China's largest economy?",1.0
156,0006ffd99a6599ff35b3,Did Julius Caesar bring a tyrannosaurus rex on his campaigns to frighten the Celts into submission?,1.0
167,00075f7061837807c69f,In what manner has Republican backing of 'states rights' been hypocritical and what ways have they actually restricted the ability of states to make their own laws?,1.0


In [6]:
df_train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province as a nation in the 1960s?,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you encourage people to adopt and not shop?",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity affect space geometry?,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg hemispheres?,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain bike by just changing the tyres?,0


In [7]:
df_train['question_text'][130757]

'What are the literary devices used in the novel "Les Misérables" by Victor Hugo?'

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(8,5))
ax = sns.barplot(x=df_train.target.unique(),y=df_train.target.value_counts(), hue = ['0', '1'], dodge = False);

#ax.set_title('Distribution of Negative & Positive Insincere question')
ax.set(xlabel= 'Target')
ax.set(ylabel= 'count')

handles, _ = ax.get_legend_handles_labels()
ax.legend(handles, ['Negative', 'Positive'])

plt.title('Distribution of Negative & Positive Insincere question')
plt.show()

<Figure size 800x500 with 1 Axes>

In [20]:
print(type(df_train['question_text']))
df_train['question_text'] = df_train.question_text.progress_apply(lambda x: re.sub('\n', ' ', x))


Progress:   0%|          | 3819/1306122 [00:00<00:34, 38189.58it/s]

<class 'pandas.core.series.Series'>


Progress: 100%|██████████| 1306122/1306122 [00:03<00:00, 405297.15it/s]


In [16]:
df_train['question_text'][0]

'How did Quebec nationalists see their province as a nation in the 1960s?'

In [17]:
nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])
def tokenizer(s): return [w.text.lower() for w in nlp(tweet_clean(s))]

In [18]:
def tweet_clean(text):
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text) # remove non alphanumeric character
    text = re.sub(r'https?:/\/\S+', ' ', text) # remove links
    return text.strip()

In [22]:
TEXT = data.Field(sequential=True, tokenize=tokenizer, include_lengths=True, use_vocab=True)
LABELS = data.Field(sequential=False, use_vocab=False, pad_token=None, unk_token=None)

train_fields = [
    ('question_text', TEXT),
    ('target', LABELS)
]

In [26]:
print(type(df_train))

<class 'list'>


In [ ]:
## seperating

In [32]:
def split_train_test(df_train, test_size=0.2):
    train, val = train_test_split(df_train, test_size=test_size,random_state=42)
    return train.reset_index(drop=True), val.reset_index(drop=True)

In [33]:
traindf, valdf = split_train_test(df_train, test_size=0.2)

AttributeError: 'list' object has no attribute 'reset_index'

In [ ]:
##crate torchtext dataset

In [29]:
%%time
trainds = data.Dataset(df_train, fields=train_fields)
valds = data.Dataset(valdf.csv',fields=train_fields, skip_header=True)

FileNotFoundError: [Errno 2] No such file or directory: './data/traindf.csv'